In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2

In [2]:
seed = 42
torch.manual_seed(seed)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")


In [4]:
from torch.utils.data import Dataset
from PIL import Image
import os

class UnlabeledDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.image_paths = [os.path.join(root, f) for f in os.listdir(root) if f.endswith(('.jpg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image  # No label

In [5]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset


transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # ✅ Ensure tensor conversion before normalization
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

target_transform = transforms.Lambda(lambda y: torch.tensor(y, dtype=torch.long))

# Load datasets
synthetic_dataset = ImageFolder(root="data/synthetic/cifar10", 
                                transform=transform_test,
                                target_transform=target_transform)
unlabeled_dataset = UnlabeledDataset("data/real/unlabelled", transform=transform_test)
test_dataset = ImageFolder(root="data/real/animal_data", transform=transform_test)

batch_size = 32
synthetic_loader = DataLoader(synthetic_dataset, batch_size=batch_size, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
class DomainDiscriminator(nn.Module):
    def __init__(self, input_size=1280):
        super(DomainDiscriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1)
        )
    def forward(self, x):
        return self.layer(x)


In [7]:
class FeatureExtractor(nn.Module):
    def __init__(self, num_classes):
        super(FeatureExtractor, self).__init__()
        mobilenet = mobilenet_v2(pretrained=True)
        self.features = mobilenet.features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, num_classes)
        )

    def forward(self, x):
        features = self.features(x)
        features = self.avgpool(features)
        features = torch.flatten(features, 1)
        class_output = self.classifier(features)
        return features, class_output

In [8]:
def train_adversarial(model, discriminator, combined_loader, unlabeled_loader, num_epochs):
    optimizer_G = optim.Adam(model.parameters(), lr=0.001)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        model.train()
        discriminator.train()
        total_loss_G, total_loss_D, total_loss_cls = 0, 0, 0
        
        unlabeled_iter = iter(unlabeled_loader)
        
        for combined_data, combined_labels in combined_loader:
            try:
                target_data = next(unlabeled_iter)
            except StopIteration:
                unlabeled_iter = iter(unlabeled_loader)
                target_data = next(unlabeled_iter)
            
            batch_size = min(combined_data.size(0), target_data.size(0))
            combined_data = combined_data[:batch_size].to(device)
            combined_labels = combined_labels[:batch_size].to(device)
            target_data = target_data[:batch_size].to(device)
            
            # Train discriminator
            optimizer_D.zero_grad()
            combined_features, combined_outputs = model(combined_data)
            target_features, _ = model(target_data)
            
            domain_pred_combined = discriminator(combined_features.detach())
            domain_pred_target = discriminator(target_features.detach())
            domain_pred_combined = domain_pred_combined.view(-1)
            domain_pred_target = domain_pred_target.view(-1)

            loss_D = -torch.mean(torch.log(torch.sigmoid(domain_pred_combined) + 1e-10) + 
                                 torch.log(1 - torch.sigmoid(domain_pred_target) + 1e-10))
            loss_D.backward()
            optimizer_D.step()
            
            # Train generator (feature extractor)
            optimizer_G.zero_grad()
            combined_features, combined_outputs = model(combined_data)
            target_features, _ = model(target_data)
            
            loss_cls = criterion(combined_outputs, combined_labels)
            domain_pred_target = discriminator(target_features)
            loss_adv = -torch.mean(torch.log(torch.sigmoid(domain_pred_target.view(-1)) + 1e-10))
            
            loss_G = loss_cls + 0.1 * loss_adv
            loss_G.backward()
            optimizer_G.step()
            
            total_loss_G += loss_G.item()
            total_loss_D += loss_D.item()
            total_loss_cls += loss_cls.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss G: {total_loss_G/len(combined_loader):.4f}, "
              f"Loss D: {total_loss_D/len(combined_loader):.4f}, "
              f"Loss Cls: {total_loss_cls/len(combined_loader):.4f}")


In [9]:
def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device))
    print(f"Model loaded from {path}")
    return model

# Load the saved model
num_classes = 3
model = FeatureExtractor(num_classes=num_classes).to(device)
model = load_model(model, 'model.pth', device)

discriminator = DomainDiscriminator().to(device)

/Users/kash/miniconda/envs/cva3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kash/miniconda/envs/cva3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded from model.pth


In [10]:
# %%
def generate_pseudo_labels(model, dataloader, threshold=0.7):
    model.eval()
    pseudo_data = []

    with torch.no_grad():
        for images in dataloader:
            images = images.to(device)
            features, outputs = model(images)  # Unpack the tuple
            probabilities = torch.softmax(outputs, dim=1)
            confidence, pseudo_labels = torch.max(probabilities, dim=1)

            for i in range(len(images)):
                if confidence[i] > threshold:
                    pseudo_data.append((images[i].cpu(), pseudo_labels[i].cpu()))
    
    return pseudo_data


class PseudoLabeledDataset(Dataset):
    def __init__(self, pseudo_data, transform=None):
        self.pseudo_data = pseudo_data
        self.transform = transform

    def __len__(self):
        return len(self.pseudo_data)

    def __getitem__(self, idx):
        image, label = self.pseudo_data[idx]
        
        # Ensure image is a tensor
        if self.transform:
            image = self.transform(image)
        
        # Ensure label is a tensor
        label = torch.tensor(label, dtype=torch.long)
        
        return image, label

In [11]:
from itertools import cycle
pseudo_data = generate_pseudo_labels(model, unlabeled_loader, threshold=0.51)
pseudo_dataset = PseudoLabeledDataset(pseudo_data)
combined_dataset = ConcatDataset([synthetic_dataset, pseudo_dataset])
combined_loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
for (combined_data, combined_labels), target_data in zip(combined_loader, cycle(unlabeled_loader)):
    batch_size = min(combined_data.size(0), target_data.size(0))
    combined_data = combined_data[:batch_size]
    combined_labels = combined_labels[:batch_size]
    target_data = target_data[:batch_size]


/tmp/ipykernel_33514/4191723487.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long)


In [12]:
train_adversarial(model, discriminator, combined_loader, unlabeled_loader, num_epochs=10)

/tmp/ipykernel_33514/4191723487.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long)


Epoch [1/10], Loss G: 0.3729, Loss D: 1.3626, Loss Cls: 0.2980
Epoch [2/10], Loss G: 0.1869, Loss D: 1.3682, Loss Cls: 0.1141
Epoch [3/10], Loss G: 0.1202, Loss D: 1.3680, Loss Cls: 0.0478
Epoch [4/10], Loss G: 0.1305, Loss D: 1.3802, Loss Cls: 0.0593
Epoch [5/10], Loss G: 0.1794, Loss D: 1.3683, Loss Cls: 0.1069
Epoch [6/10], Loss G: 0.1395, Loss D: 1.3671, Loss Cls: 0.0662
Epoch [7/10], Loss G: 0.1805, Loss D: 1.3693, Loss Cls: 0.1059
Epoch [8/10], Loss G: 0.1720, Loss D: 1.3599, Loss Cls: 0.0976
Epoch [9/10], Loss G: 0.1593, Loss D: 1.3637, Loss Cls: 0.0862
Epoch [10/10], Loss G: 0.1826, Loss D: 1.3663, Loss Cls: 0.1090


In [14]:
current_seed = torch.initial_seed()
print(f"Current seed: {current_seed}")

Current seed: 42


In [13]:
model.eval()
correct = 0
total = 0
test_loss = 0.0
criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        _, outputs = model(images)
        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        test_loss += loss.item()

test_accuracy = 100 * correct / total
test_loss /= len(test_loader)

print(f"\nFinal Test Accuracy: {test_accuracy:.2f}% | Test Loss: {test_loss:.4f}")

# Save the improved model
torch.save(model.state_dict(), "domain_adapted_model.pth")
print("\nImproved model saved as 'domain_adapted_model.pth'.")


Final Test Accuracy: 82.46% | Test Loss: 0.5370

Improved model saved as 'domain_adapted_model.pth'.
